In [ ]:
import pandas as pd
import sys
import os
import clip as clip
import wandb
wandb.login()

%reload_ext autoreload
%autoreload 2

# Data Loading

In [3]:
img = pd.read_table(f'...', sep='\t', index_col=0)
ppi = pd.read_table(f'..', sep='\t', index_col=0)

In [4]:
overlapping_proteins = list(set(img.index.values).intersection(set(ppi.index.values)))
print(len(overlapping_proteins))

876


In [5]:
img = img.loc[overlapping_proteins]
ppi = ppi.loc[overlapping_proteins]

# ModelTrain

In [ ]:
wandb.finish()
img_features = img
ppi_features = ppi

wandb.init(
    # Set the project where this run will be logged
    project="contrastive_learning",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"exp-1",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 1e-4,
        "model": "ClipModel",
        "epochs": 100,
        "batch_size": 128,
        "dropout": 0.1,  # 0.25
        "latent_dim": 128,  # output dim of embedding
        "hidden_dim": 512,
        "imgdim": img_features.shape[1],  # input dim of image
        "ppidim": ppi_features.shape[1],  # input dim of ppi
        "n_sample": img_features.shape[0],
    },
)

In [ ]:
resultsdir = f"..." # where to save the embeddings and model artifact
if not os.path.exists(resultsdir):
    os.makedirs(resultsdir, exist_ok=True)
    
model = clip.clip_fit_predict(
    resultsdir=resultsdir,
    index_names=overlapping_proteins,
    data_ppi=ppi_features.values,
    data_img=img_features.values,
    config=wandb.config,
    wandb=wandb,
)
wandb.finish()